In [1]:
import pandas as pd
import numpy as np


In [2]:
filelist = ['birth.csv', 'breed.csv', 'report.csv', 'spec.csv','submission.csv']

# Read File

In [3]:
birth = pd.read_csv('./data/' + filelist[0])
breed = pd.read_csv('./data/' + filelist[1])
report = pd.read_csv('./data/' + filelist[2])
spec = pd.read_csv('./data/' + filelist[3])
submission = pd.read_csv('./data/' + filelist[4])
submission

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,1
0,480,NaN
1,742,NaN
2,743,NaN
3,744,NaN
4,745,NaN
...,...,...
4258,37481,NaN
4259,37482,NaN
4260,37483,NaN
4261,37484,NaN


In [4]:
birth_index = ['乳牛編號', '分娩日期', '乾乳日期','犢牛編號1','犢牛編號2','母牛體重','登錄日期','計算胎次','胎次','分娩難易度','犢牛體型','犢牛性別','酪農場代號']
report_index = ['ID','資料年度','資料月份','酪農場代號','乳牛編號','父親精液編號','母親乳牛編號','出生日期','胎次','泌乳天數','乳量','最近分娩日期','採樣日期','月齡','檢測日期','最後配種日期','最後配種精液','配種次數','前次分娩日期','第一次配種日期','第一次配種精液']
breed_index = ['乳牛編號','配種日期','配種精液','登錄日期','孕檢','配種方式','精液種類','酪農場代號']
spec_index = ['乳牛編號','狀況類別','狀況代號','狀況日期','備註','登錄日期','酪農場代號']
birth.columns = birth_index
report.columns = report_index
breed.columns = breed_index
spec.columns = spec_index
# birth.fillna(0,inplace = True)
# report.fillna(0,inplace = True)
# breed.fillna(0, inplace = True)
# spec.fillna(0, inplace = True)
report.head()

,ID,資料年度,資料月份,酪農場代號,乳牛編號,父親精液編號,母親乳牛編號,出生日期,胎次,泌乳天數,...,最近分娩日期,採樣日期,月齡,檢測日期,最後配種日期,最後配種精液,配種次數,前次分娩日期,第一次配種日期,第一次配種精液
0,1,2013,3,A,52612,507H7173,96040545,2011/1/1 00:00,1,78.0,...,2013/1/1 00:00,2013/3/20 00:00,26,2013/3/21 00:00,2015/5/5 00:00,507H11621,1,NaN,2013/3/5 00:00,501H8784
1,2,2013,4,A,52612,507H7173,96040545,2011/1/1 00:00,1,107.0,...,2013/1/1 00:00,2013/4/18 00:00,27,2013/4/19 00:00,2015/5/5 00:00,507H11621,1,NaN,2013/3/5 00:00,501H8784
2,3,2013,5,A,52612,507H7173,96040545,2011/1/1 00:00,1,140.0,...,2013/1/1 00:00,2013/5/21 00:00,28,2013/5/22 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784
3,4,2013,6,A,52612,507H7173,96040545,2011/1/1 00:00,1,175.0,...,2013/1/1 00:00,2013/6/25 00:00,29,2013/6/26 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784
4,5,2013,7,A,52612,507H7173,96040545,2011/1/1 00:00,1,210.0,...,2013/1/1 00:00,2013/7/30 00:00,30,2013/7/31 00:00,2015/5/5 00:00,507H11621,2,NaN,2013/3/5 00:00,501H8784


# 日期處理

In [5]:
def augFeatures(data, name):
    data[name] = pd.to_datetime(data[name])
#     data[name+"年"] = data[name].dt.year
    data[name+'月'] = data[name].dt.month
#     data[name+'日'] = data[name].dt.day
#     train[name+'星期'] = train[name].dt.dayofweek
    return data

In [6]:
# report = augFeatures(report, '最近分娩日期')

# tx = pd.get_dummies(report['最近分娩日期月'], prefix = '最近分娩日期月')
# report = pd.concat([report, tx], axis = 1)
# report = report.drop(columns=['最近分娩日期月'], axis = 1)


# Processing NA

In [7]:
report['乳量'] = report.groupby(by=['乳牛編號','胎次'])['乳量'].transform(lambda x: x.fillna(x.mean()))

report['泌乳天數'] = report.groupby(by=['乳牛編號','胎次'])['泌乳天數'].transform(lambda x: x.fillna(x.mean()))

birth['母牛體重'] = birth.groupby(by=['乳牛編號'])['母牛體重'].transform(lambda x: x.fillna(x.mean()))
birth['分娩難易度'] = birth.groupby(by=['乳牛編號'])['分娩難易度'].transform(lambda x: x.fillna(x.mean()))

# 建立時序資料

In [8]:
# SQL
def buildtable(data):
    t1 = data
    t2 = data
    t3 = data
    # Join
    t4 = pd.merge(t1, t2, on = ['乳牛編號', '胎次'], suffixes = ['_L', '_R'])
    # Filter
    t5 = t4[(t4.月齡_L >= t4.月齡_R)]
    # Group by
    t6 = t5.groupby(['ID_L'], as_index = False)
    # Aggregate
    t7 = t6.agg({'ID_R': 'count'})
    
    t8 = t7.rename(columns = {'ID_L': 'ID', 'ID_R': '序列號'})
    t9 = pd.merge(t8, t3, on = ['ID'])
    t10 = t9.sort_values(by = ['乳牛編號', '胎次', '序列號'])
    return t10


# build time series
def build_data(train, pastMonth=5, futureMonth=5):
    train['過去乳量-1'] = train.groupby(by=['乳牛編號', '胎次'])['乳量'].shift()
    for i in range(2, pastMonth+1):
        train['過去乳量-%d'%(i)] = train.groupby(by=['乳牛編號', '胎次'])['過去乳量-%d'%(i-1)].shift()
    train['未來乳量-1'] = train.groupby(by=['乳牛編號', '胎次'])['乳量'].shift(-1)
    for i in range(2, futureMonth+1):
        train['未來乳量-%d'%(i)] = train.groupby(by=['乳牛編號', '胎次'])['未來乳量-%d'%(i-1)].shift(-1)
    return train

In [9]:
# report = buildtable(report)
# report = build_data(report, pastMonth=20, futureMonth=1)

# 新feature Spec 狀況

In [10]:
report['不健康'] = 0
from datetime import datetime
for i in range(len(spec)):
    date = datetime.strptime(spec['狀況日期'][i], "%Y/%m/%d %H:%M")
    if len(report.index[report['乳牛編號'] == spec['乳牛編號'][i]]):
        for j in report.index[report['乳牛編號'] == spec['乳牛編號'][i]]:
            if report['資料年度'][j] == date.year and report['資料月份'][j] == date.month and report['酪農場代號'][j]==spec['酪農場代號'][i]:
                report.at[j, '不健康'] = 1

# One Hot Encode report

In [11]:
tx = pd.get_dummies(report['酪農場代號'], prefix = '酪農場代號')
report = pd.concat([report, tx], axis = 1)

tx = pd.get_dummies(report['資料月份'], prefix = '資料月份')
report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['乳牛編號'], prefix = '乳牛編號')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['父親精液編號'], prefix = '父親精液編號')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['母親乳牛編號'], prefix = '母親乳牛編號')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['出生日期'], prefix = '出生日期')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['最近分娩日期'], prefix = '最近分娩日期')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['最後配種日期'], prefix = '最後配種日期')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['採樣日期'], prefix = '採樣日期')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['檢測日期'], prefix = '檢測日期')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['最後配種精液'], prefix = '最後配種精液')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['前次分娩日期'], prefix = '前次分娩日期')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['第一次配種日期'], prefix = '第一次配種日期')
# report = pd.concat([report, tx], axis = 1)

# tx = pd.get_dummies(report['第一次配種精液'], prefix = '第一次配種精液')
# report = pd.concat([report, tx], axis = 1)

In [12]:
Train_ID = [rep_val for rep_val in report['ID'].values if rep_val not in submission['ID'].values]
Train_report = report.loc[report['ID'].isin(Train_ID)]
Train_Number = Train_report['乳牛編號'].values
Train_report['ID'].values

array([    1,     2,     3, ..., 37093, 37092, 37089], dtype=int64)

In [13]:
Test_ID = [rep_val for rep_val in report['ID'].values if rep_val in submission['ID'].values]
Test_report = report.loc[report['ID'].isin(Test_ID)]
Test_Number = Test_report['乳牛編號'].values
Test_report

,ID,序列號,資料年度,資料月份,酪農場代號,乳牛編號,父親精液編號,母親乳牛編號,出生日期,胎次,...,資料月份_3,資料月份_4,資料月份_5,資料月份_6,資料月份_7,資料月份_8,資料月份_9,資料月份_10,資料月份_11,資料月份_12
479,480,15,2019,1,A,52631,501H9952,97040152,2011/3/17 00:00,4,...,0,0,0,0,0,0,0,0,0,0
741,742,1,2019,2,A,52640,501H9952,96040567,2011/4/1 00:00,4,...,0,0,0,0,0,0,0,0,0,0
742,743,2,2019,3,A,52640,501H9952,96040567,2011/4/1 00:00,4,...,1,0,0,0,0,0,0,0,0,0
743,744,3,2019,4,A,52640,501H9952,96040567,2011/4/1 00:00,4,...,0,1,0,0,0,0,0,0,0,0
744,745,4,2019,5,A,52640,501H9952,96040567,2011/4/1 00:00,4,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37026,37027,4,2019,3,C,99183209,514H4148,9.51704e+07,2010/7/16 00:00,6,...,1,0,0,0,0,0,0,0,0,0
37027,37028,5,2019,4,C,99183209,514H4148,9.51704e+07,2010/7/16 00:00,6,...,0,1,0,0,0,0,0,0,0,0
37029,37030,6,2019,5,C,99183209,514H4148,9.51704e+07,2010/7/16 00:00,6,...,0,0,1,0,0,0,0,0,0,0
37028,37029,7,2019,6,C,99183209,514H4148,9.51704e+07,2010/7/16 00:00,6,...,0,0,0,1,0,0,0,0,0,0


In [14]:
report_drop_columns = ['ID','資料年度','資料月份','父親精液編號','母親乳牛編號','出生日期','最近分娩日期','採樣日期','檢測日期'
               ,'最後配種日期','最後配種精液','前次分娩日期','第一次配種日期','第一次配種精液', '酪農場代號','月齡']
Test_report_drop_columns = ['資料年度','資料月份','父親精液編號','母親乳牛編號','出生日期','最近分娩日期','採樣日期','檢測日期'
               ,'最後配種日期','最後配種精液','前次分娩日期','第一次配種日期','第一次配種精液', '酪農場代號', '月齡']
Train_report_x = Train_report.drop(columns=report_drop_columns, axis = 1)

Test_report_x = Test_report.drop(columns=Test_report_drop_columns, axis = 1)
Test_report_x

,ID,序列號,乳牛編號,胎次,泌乳天數,乳量,配種次數,過去乳量-1,過去乳量-2,過去乳量-3,...,資料月份_3,資料月份_4,資料月份_5,資料月份_6,資料月份_7,資料月份_8,資料月份_9,資料月份_10,資料月份_11,資料月份_12
479,480,15,52631,4,435.0,28.021429,6,15.8,20.6,25.3,...,0,0,0,0,0,0,0,0,0,0
741,742,1,52640,4,13.0,NaN,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
742,743,2,52640,4,35.0,NaN,1,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
743,744,3,52640,4,70.0,NaN,1,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
744,745,4,52640,4,98.0,NaN,1,NaN,NaN,NaN,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37026,37027,4,99183209,6,110.0,10.000000,2,10.0,10.0,10.0,...,1,0,0,0,0,0,0,0,0,0
37027,37028,5,99183209,6,141.0,10.000000,3,10.0,10.0,10.0,...,0,1,0,0,0,0,0,0,0,0
37029,37030,6,99183209,6,169.0,10.000000,3,10.0,10.0,10.0,...,0,0,1,0,0,0,0,0,0,0
37028,37029,7,99183209,6,202.0,10.000000,3,10.0,10.0,10.0,...,0,0,0,1,0,0,0,0,0,0


In [15]:
# # breed: one hot encode 
# tx = pd.get_dummies(breed['配種方式'], prefix = '配種方式')
# breed = pd.concat([breed, tx], axis = 1)
# tx = pd.get_dummies(breed['精液種類'], prefix = '精液種類')
# breed = pd.concat([breed, tx], axis = 1)

# breed_drop_columns = ['配種日期','配種精液','登錄日期','孕檢', '配種方式', '精液種類', '酪農場代號']
# breed_x = breed.drop(columns=breed_drop_columns, axis = 1)

# Train_breed_x = breed_x.loc[breed_x['乳牛編號'].isin(Train_Number)]
# Test_breed_x = breed_x.loc[breed_x['乳牛編號'].isin(Test_Number)]
# Test_breed_x

In [16]:
# birth feature
birth_drop_columns = ['分娩日期', '乾乳日期', '犢牛編號1', '犢牛編號2', '登錄日期', '計算胎次', '犢牛體型', '犢牛性別', '酪農場代號']
birth_x = birth.drop(columns=birth_drop_columns, axis = 1)

Train_birth_x = birth_x.loc[birth_x['乳牛編號'].isin(Train_Number)]
Test_birth_x = birth_x.loc[birth_x['乳牛編號'].isin(Test_Number)]
Train_birth_x

,乳牛編號,母牛體重,胎次,分娩難易度
0,87121677,NaN,2,1.0
1,96126940,522.0,2,2.0
2,95181708,670.0,5,1.0
3,97127665,NaN,3,2.0
4,97182634,630.0,4,1.0
...,...,...,...,...
3755,4181793,590.0,2,3.0
3756,5184341,560.0,2,1.0
3757,5184348,580.0,2,1.0
3758,97182634,730.0,9,1.0


In [17]:
# # 備註特徵轉換 問題牛
# bad = {'腳痛', '繁殖障礙', '內寄生蟲', '死亡', '難產', '木乃伊', '無法站立', '蹄的問題', '其他疾病', '中毒', '2種原因以上'
#       , '感冒', '腳斷', '淘汰', '代謝疾病', '生殖道問題', '有抗生素'}

# spec['備註'] = spec['備註'].astype(str)
# for i in range(len(spec['備註'])):
#     spec['備註'][i] = '1' if spec['備註'][i] in bad else '0'
# spec['備註'] = spec['備註'].astype(int)

# # # one hot encode
# # tx = pd.get_dummies(spec['狀況類別'], prefix = '狀況類別')
# # spec = pd.concat([spec, tx], axis = 1)

# spec_drop_columns = ['狀況類別','狀況代號','狀況日期','登錄日期','酪農場代號']
# spec_x = spec.drop(columns=spec_drop_columns, axis = 1)

# Train_spec_x = spec_x.loc[spec_x['乳牛編號'].isin(Train_Number)]
# Test_spec_x = spec_x.loc[spec_x['乳牛編號'].isin(Test_Number)]
# Train_spec_x

In [18]:
# def season(time):
#     if len(item) == 0:
#         return 0
#     temp = int(item.split('/')[1])
#     if temp >= 1 and temp <= 3:
#         return '春'
#     elif temp >= 4 and temp <= 6:
#         return '夏'
#     elif temp >= 7 and temp <= 9:
#         return '秋'
#     elif temp >= 10 and temp <= 12:
#         return '冬'
#     else:
#         return 0

In [19]:
# days = Train_report_x['泌乳天數'].mean()
# Train_report_x['泌乳天數'].fillna(days, inplace = True)

# weight = Train_birth_x['母牛體重'].mean()
# Train_birth_x['母牛體重'].fillna(weight, inplace = True)

# weight = Test_birth_x['母牛體重'].mean()
# Test_birth_x['母牛體重'].fillna(weight, inplace = True)

# hard = Train_birth_x['分娩難易度'].mean()
# Train_birth_x['分娩難易度'].fillna(hard, inplace = True)

# hard = Train_birth_x['分娩難易度'].mean()
# Test_birth_x['分娩難易度'].fillna(hard, inplace = True)

# Data Normalization

In [20]:
def normalize(data):
    data_norm = data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return data_norm


birth_normal = ['母牛體重', '胎次', '分娩難易度']
for normal_col in birth_normal:
    try:
        normalized_x = (Train_birth_x[normal_col]-Train_birth_x[normal_col].mean()) / Train_birth_x[normal_col].std()
        Train_birth_x[normal_col] = normalized_x

        normalized_x = (Test_birth_x[normal_col]-Test_birth_x[normal_col].mean()) / Test_birth_x[normal_col].std()
        Test_birth_x[normal_col] = normalized_x
    except:
        print(f'{normal_col} not found')
    
report_normal = ['泌乳天數', '胎次', '月齡', '配種次數']
for normal_col in report_normal:
    try:
        normalized_x = (Train_report_x[normal_col]-Train_report_x[normal_col].mean()) / Train_report_x[normal_col].std()
        Train_report_x[normal_col] = normalized_x

        normalized_x = (Test_report_x[normal_col]-Test_report_x[normal_col].mean()) / Test_report_x[normal_col].std()
        Test_report_x[normal_col] = normalized_x
    except:
        print(f'{normal_col} not found')

月齡 not found


D:\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


# Merge Data

In [21]:
x = pd.merge(Train_report_x, Train_birth_x, on= ['乳牛編號'])

# x = pd.merge(x, Train_breed_x, on = ['乳牛編號'])
# x = pd.merge(x, Train_spec_x, on= ['乳牛編號'])

y = x['乳量'].values
x = x.drop(columns = ['乳牛編號','乳量'], axis = 1)
x = x.fillna(-1)

# x = normalize(x)
x

,序列號,胎次_x,泌乳天數,配種次數,過去乳量-1,過去乳量-2,過去乳量-3,過去乳量-4,過去乳量-5,過去乳量-6,...,資料月份_6,資料月份_7,資料月份_8,資料月份_9,資料月份_10,資料月份_11,資料月份_12,母牛體重,胎次_y,分娩難易度
0,1,-0.778490,-0.853635,-0.454949,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,0,0,0,0,0,-1.000000,-0.868055,-1.000000
1,1,-0.778490,-0.853635,-0.454949,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,0,0,0,0,0,-1.000000,-0.086893,-1.000000
2,2,-0.778490,-0.660737,-0.454949,29.4,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,0,0,0,0,0,-1.000000,-0.868055,-1.000000
3,2,-0.778490,-0.660737,-0.454949,29.4,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,0,0,0,0,0,-1.000000,-0.086893,-1.000000
4,3,-0.778490,-0.441233,0.014484,27.8,29.4,-1.0,-1.0,-1.0,-1.0,...,0,0,0,0,0,0,0,-1.000000,-0.868055,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87181,6,0.855796,-0.215077,0.014484,18.0,23.0,23.0,20.0,22.0,-1.0,...,0,0,0,0,0,0,0,0.969423,0.694270,-0.526305
87182,7,0.855796,-0.008876,0.014484,18.0,18.0,23.0,23.0,20.0,22.0,...,0,0,0,0,0,0,0,0.195929,-0.086893,-0.526305
87183,7,0.855796,-0.008876,0.014484,18.0,18.0,23.0,23.0,20.0,22.0,...,0,0,0,0,0,0,0,0.969423,0.694270,-0.526305
87184,8,0.855796,0.190673,0.014484,15.0,18.0,18.0,23.0,23.0,20.0,...,0,0,0,0,0,0,0,0.195929,-0.086893,-0.526305


In [22]:
test = pd.merge(Test_report_x, Test_birth_x, on =  ['乳牛編號'], how = 'left')

# test = pd.merge(test, Test_breed_x, on =  ['乳牛編號'], how = 'left')
# test = pd.merge(test, Test_spec_x, on =  ['乳牛編號'], how = 'left')
test.fillna(-1, inplace = True)

test_without_ID = test.drop(columns = ['乳牛編號','乳量','ID'], axis = 1)

# test_without_ID = normalize(test_without_ID)
# test_without_ID.fillna(0, inplace = True)
test_without_ID

,序列號,胎次_x,泌乳天數,配種次數,過去乳量-1,過去乳量-2,過去乳量-3,過去乳量-4,過去乳量-5,過去乳量-6,...,資料月份_6,資料月份_7,資料月份_8,資料月份_9,資料月份_10,資料月份_11,資料月份_12,母牛體重,胎次_y,分娩難易度
0,15,1.636775,1.671192,1.995674,15.8,20.6,25.3,19.8,24.6,26.0,...,0,0,0,0,0,0,0,-1.000000,-0.765127,-1.000000
1,15,1.636775,1.671192,1.995674,15.8,20.6,25.3,19.8,24.6,26.0,...,0,0,0,0,0,0,0,-1.000000,0.076127,-1.000000
2,15,1.636775,1.671192,1.995674,15.8,20.6,25.3,19.8,24.6,26.0,...,0,0,0,0,0,0,0,-1.000000,0.917382,-1.000000
3,15,1.636775,1.671192,1.995674,15.8,20.6,25.3,19.8,24.6,26.0,...,0,0,0,0,0,0,0,-1.000000,1.758636,-1.000000
4,1,1.636775,-1.251356,-0.924708,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,0,0,0,0,0,0,0,-1.000000,-0.765127,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8825,8,3.284760,0.265322,0.535483,10.0,10.0,10.0,10.0,10.0,10.0,...,0,1,0,0,0,0,0,-0.110320,0.076127,-0.488518
8826,8,3.284760,0.265322,0.535483,10.0,10.0,10.0,10.0,10.0,10.0,...,0,1,0,0,0,0,0,0.451652,0.917382,-0.488518
8827,8,3.284760,0.265322,0.535483,10.0,10.0,10.0,10.0,10.0,10.0,...,0,1,0,0,0,0,0,1.107285,1.758636,-0.488518
8828,8,3.284760,0.265322,0.535483,10.0,10.0,10.0,10.0,10.0,10.0,...,0,1,0,0,0,0,0,1.762919,2.599891,-0.488518
